# Example of a PAGES2k query and data transformation

For integration into ClimateDisk, we need to be able to query from the LinkedEarth GraphDB directly and create a pickle file that cfr will like. 

In [1]:
import json
import requests
import wget
import lipd as lpd
import pandas as pd
import io
import ast
import numpy as np

## Example pickle file

In [21]:
df_pickle = pd.read_pickle('data/pages2k_dataset.pkl')
df_pickle

,paleoData_pages2kID,dataSetName,archiveType,geo_meanElev,geo_meanLat,geo_meanLon,year,yearUnits,paleoData_variableName,paleoData_units,paleoData_values,paleoData_proxy
0,NAm_153,NAm-Bennington.Luckman.2001,tree,1850.0,52.7,-118.3,"[1563.0, 1564.0, 1565.0, 1566.0, 1567.0, 1568....",AD,trsgi,NA,"[0.883, 0.942, 1.043, 0.945, 1.141, 0.715, 0.6...",TRW
1,Asi_245,Asi-SouthChina.Wang.1998,documents,2200.0,23.0,114.0,"[1500.0, 1510.0, 1520.0, 1530.0, 1540.0, 1550....",AD,temperature,index,"[0.157429639092533, -0.118646665464906, 0.3035...",historic
2,NAm_165,NAm-RedMountainPassSilverton.Graybill.1994,tree,3400.0,37.9,-107.7,"[1626.0, 1627.0, 1628.0, 1629.0, 1630.0, 1631....",AD,MXD,NA,"[1.071, 1.014, 1.014, 0.966, 1.005, 1.072, 1.0...",MXD
3,Asi_178,Asi-BURGPW.PAGES2k.2013,tree,2500.0,28.77,83.73,"[1303.0, 1304.0, 1305.0, 1306.0, 1307.0, 1308....",AD,trsgi,NA,"[0.889, 0.788, 0.836, 0.741, 0.613, 0.732, 0.7...",TRW
4,Asi_174,Asi-NEPA029.Krusic.2013,tree,4000.0,28.18,85.43,"[1559.0, 1560.0, 1561.0, 1562.0, 1563.0, 1564....",AD,trsgi,NA,"[1.172, 1.21, 1.145, 1.307, 1.368, 1.37, 1.152...",TRW
...,...,...,...,...,...,...,...,...,...,...,...,...
687,Asi_201,Asi-PAKI030.Cook.2013,tree,3630.0,35.88,74.18,"[1574.0, 1575.0, 1576.0, 1577.0, 1578.0, 1579....",AD,trsgi,NA,"[1.212, 0.873, 0.859, 0.968, 0.737, 0.956, 0.8...",TRW
688,Asi_179,Asi-NEPA021.Krusic.2013,tree,3400.0,27.5,88.02,"[1525.0, 1526.0, 1527.0, 1528.0, 1529.0, 1530....",AD,trsgi,NA,"[1.316, 1.139, 1.224, 1.416, 1.305, 1.259, 1.2...",TRW
689,Arc_014,Arc-LakeLehmilampi.Haltia-Hovi.2007,lake sediment,95.0,63.62,29.1,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",AD,thickness,mm,"[-1.3, -0.84, -0.43, -0.69, -0.51, -0.51, -0.5...",varve thickness
690,Ocn_071,Ocn-Guadeloupe.Steinhilber.2010,coral,-1.7,16.2,-61.49,"[1999.21, 1999.13, 1999.04, 1998.96, 1998.88, ...",AD,d18O,permil,"[-4.083, -4.08167, -4.08033, -4.079, -4.24967,...",d18O


Let's get the name of the columns

In [87]:
list(df_pickle.columns)

['paleoData_pages2kID',
 'dataSetName',
 'archiveType',
 'geo_meanElev',
 'geo_meanLat',
 'geo_meanLon',
 'year',
 'yearUnits',
 'paleoData_variableName',
 'paleoData_units',
 'paleoData_values',
 'paleoData_proxy']

In [17]:
df_pickle[df_pickle['dataSetName']=='Ant-WDC05A.Steig.2013']

,paleoData_pages2kID,dataSetName,archiveType,geo_meanElev,geo_meanLat,geo_meanLon,year,yearUnits,paleoData_variableName,paleoData_units,paleoData_values,paleoData_proxy
158,Ant_006,Ant-WDC05A.Steig.2013,glacier ice,1806.0,-79.46,-112.09,"[2005.0, 2004.0, 2003.0, 2002.0, 2001.0, 2000....",AD,d18O,permil,"[-33.32873325, -35.6732, -33.1574, -34.2854, -...",d18O


## Using the graphDatabase

Create the query (note some fields have been altered following legacy code from the original LinkedEarth work; we need to fix this):

In [13]:
url = 'https://linkedearth.graphdb.mint.isi.edu/repositories/LiPDVerse'

query = """PREFIX le: <http://linked.earth/ontology#>
select ?dsname ?val ?timeval ?archive ?pages2kID ?timeunits ?varname ?varunits ?lat ?lon ?alt ?proxysensor ?proxyobs FROM <http://linked.earth/lipd/Pages2k2_1_2> where { 
	?ds  le:includesPaleoData ?data .
    ?ds le:name ?dsname .
    ?ds le:hasUrl ?url .
    ?ds  le:collectedFrom ?loc .
    ?loc le:wgs84_Lat ?lat .
    ?loc le:wgs84_Long ?lon .
    ?loc le:wgs84_Alt ?alt .
	?data le:foundInMeasurementTable ?table .
	?table le:includesVariable ?var .
    ?var le:useInGlobalTemperatureAnalysis true .
    ?var le:hasValues ?val .
    ?var le:name ?varname . 
    ?var le:hasUnits ?varunits .
    ?table le:includesVariable ?timevar .
    ?timevar le:name "year" .
    ?timevar le:hasValues ?timeval .
    ?ds le:proxyArchiveType ?archive .
    ?timevar le:hasUnits ?timeunits .
    ?var le:hasProxySystem ?proxysystem .
    ?proxysystem le:proxySensorType ?proxysensor .
    ?proxysystem le:proxyObservationType ?proxyobs .
    ?var le:pages2kID ?pages2kID .
}"""

In [14]:
response = requests.post(url, data = {'query': query})

In [15]:
data = io.StringIO(response.text)
df = pd.read_csv(data, sep=",")

df

,dsname,val,timeval,archive,pages2kID,timeunits,varname,varunits,lat,lon,alt,proxysensor,proxyobs
0,Ant-WDC05A.Steig.2013,"[-33.32873325, -35.6732, -33.1574, -34.2854, -...","[2005, 2004, 2003, 2002, 2001, 2000, 1999, 199...",glacier ice,Ant_006,AD,d18O,permil,-79.460,-112.090,1806.0,D18ODefaultSensor,D18O
1,Ocn-Mayotte.Zinke.2008,"[-4.72, -4.95, -5.45, -5.157, -5.05, -5.065, -...","[1993.62, 1993.46, 1993.29, 1993.12, 1992.96, ...",coral,Ocn_139,AD,d18O,permil,-12.650,45.100,-3.0,Porites porites lutea,D18O
2,Ocn-Mayotte.Zinke.2008,"[8.6, 8.701, 8.802, 8.84, 8.877, 8.804, 8.588,...","[1994.29, 1994.12, 1993.96, 1993.79, 1993.62, ...",coral,Ocn_061,AD,Sr_Ca,mmol/mol,-12.650,45.100,-3.0,Porites porites lutea,Sr/Ca
3,Ocn-Mayotte.Zinke.2008,"[0.534, 0.535, 0.721, 0.614, 0.458, 0.033, 0.3...","[1993.62, 1993.46, 1993.29, 1993.12, 1992.96, ...",coral,Ocn_062,AD,d18O,permil,-12.650,45.100,-3.0,Porites porites lutea,D18O
4,Ocn-LosRoques.Hetzinger.2008,"[-3.8123, -3.73082, -3.74912, -3.6656, -3.6995...","[1917.79, 1917.87, 1917.96, 1918.04, 1918.12, ...",coral,Ocn_147,AD,d18O,permil,11.770,-66.750,-2.0,Diploria diploria strigosa,D18O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Ocn-NorthEastBreakers_Bermuda.Kuhnert.2005,"[-4.15, -3.66, -3.69, -4.07, -3.95, -4.12, -3....","[1983.21, 1983.13, 1983.04, 1982.96, 1982.88, ...",coral,Ocn_068,AD,d18O,permil,32.467,-64.700,-12.0,Diploria diploria strigosa,D18O
103,Ocn-NorthEastBreakers_Bermuda.Kuhnert.2005,"[9.08, 9.17, 9.14, 9.14, 9.17, 9.09, 9.08, 9.1...","[1983.21, 1983.13, 1983.04, 1982.96, 1982.88, ...",coral,Ocn_069,AD,Sr_Ca,mmol/mol,32.467,-64.700,-12.0,Diploria diploria strigosa,Sr/Ca
104,Afr-ColdAirCave.Sundqvist.2013,"[0.459055, 0.4062401, 0.3621507, 0.3517711, 0....","[1992, 1991, 1990, 1989, 1988, 1987, 1986, 198...",speleothem,Afr_012,AD,d18O,permil,-24.000,29.180,1450.0,D18ODefaultSensor,D18O
105,Ocn-TongueoftheOcean.Rosenheim.2005,"[0.356, -0.053, 0.639, 0.714, 0.6, 0.586, 0.58...","[1890, 1891, 1892, 1893, 1894, 1895, 1896, 189...",sclerosponge,Ocn_142,AD,d18O,permil,23.504,-76.577,-143.0,Ceratoporella ceratoporella nicholsoni,D18O


Let's fix the dataframe

In [22]:
df['val']=df['val'].apply(lambda row : ast.literal_eval(row))
df['timeval']=df['timeval'].apply(lambda row : ast.literal_eval(row))

df

,dsname,val,timeval,archive,pages2kID,timeunits,varname,varunits,lat,lon,alt,proxysensor,proxyobs
0,Ant-WDC05A.Steig.2013,"[-33.32873325, -35.6732, -33.1574, -34.2854, -...","[2005, 2004, 2003, 2002, 2001, 2000, 1999, 199...",glacier ice,Ant_006,AD,d18O,permil,-79.460,-112.090,1806.0,D18ODefaultSensor,D18O
1,Ocn-Mayotte.Zinke.2008,"[-4.72, -4.95, -5.45, -5.157, -5.05, -5.065, -...","[1993.62, 1993.46, 1993.29, 1993.12, 1992.96, ...",coral,Ocn_139,AD,d18O,permil,-12.650,45.100,-3.0,Porites porites lutea,D18O
2,Ocn-Mayotte.Zinke.2008,"[8.6, 8.701, 8.802, 8.84, 8.877, 8.804, 8.588,...","[1994.29, 1994.12, 1993.96, 1993.79, 1993.62, ...",coral,Ocn_061,AD,Sr_Ca,mmol/mol,-12.650,45.100,-3.0,Porites porites lutea,Sr/Ca
3,Ocn-Mayotte.Zinke.2008,"[0.534, 0.535, 0.721, 0.614, 0.458, 0.033, 0.3...","[1993.62, 1993.46, 1993.29, 1993.12, 1992.96, ...",coral,Ocn_062,AD,d18O,permil,-12.650,45.100,-3.0,Porites porites lutea,D18O
4,Ocn-LosRoques.Hetzinger.2008,"[-3.8123, -3.73082, -3.74912, -3.6656, -3.6995...","[1917.79, 1917.87, 1917.96, 1918.04, 1918.12, ...",coral,Ocn_147,AD,d18O,permil,11.770,-66.750,-2.0,Diploria diploria strigosa,D18O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Ocn-NorthEastBreakers_Bermuda.Kuhnert.2005,"[-4.15, -3.66, -3.69, -4.07, -3.95, -4.12, -3....","[1983.21, 1983.13, 1983.04, 1982.96, 1982.88, ...",coral,Ocn_068,AD,d18O,permil,32.467,-64.700,-12.0,Diploria diploria strigosa,D18O
103,Ocn-NorthEastBreakers_Bermuda.Kuhnert.2005,"[9.08, 9.17, 9.14, 9.14, 9.17, 9.09, 9.08, 9.1...","[1983.21, 1983.13, 1983.04, 1982.96, 1982.88, ...",coral,Ocn_069,AD,Sr_Ca,mmol/mol,32.467,-64.700,-12.0,Diploria diploria strigosa,Sr/Ca
104,Afr-ColdAirCave.Sundqvist.2013,"[0.459055, 0.4062401, 0.3621507, 0.3517711, 0....","[1992, 1991, 1990, 1989, 1988, 1987, 1986, 198...",speleothem,Afr_012,AD,d18O,permil,-24.000,29.180,1450.0,D18ODefaultSensor,D18O
105,Ocn-TongueoftheOcean.Rosenheim.2005,"[0.356, -0.053, 0.639, 0.714, 0.6, 0.586, 0.58...","[1890, 1891, 1892, 1893, 1894, 1895, 1896, 189...",sclerosponge,Ocn_142,AD,d18O,permil,23.504,-76.577,-143.0,Ceratoporella ceratoporella nicholsoni,D18O


Only look for the included in global database

In [25]:
url = 'https://linkedearth.graphdb.mint.isi.edu/repositories/LiPDVerse'

query = """PREFIX le: <http://linked.earth/ontology#>
select ?dsname FROM <http://linked.earth/lipd/Pages2k2_1_2> where { 
	?ds  le:includesPaleoData ?data .
    OPTIONAL {?ds le:name ?dsname .}
	?data le:foundInMeasurementTable ?table .
	?table le:includesVariable ?var .
    ?var le:useInGlobalTemperatureAnalysis true .
}"""


response = requests.post(url, data = {'query': query})

data = io.StringIO(response.text)
df = pd.read_csv(data, sep=",")

In [26]:
df

,dsname
0,Ant-WDC05A.Steig.2013
1,Arc-Arjeplog.Bjorklund.2014
2,Asi-CHIN019.Li.2010
3,Asi-GANGCD.PAGES2k.2013
4,Ocn-Mayotte.Zinke.2008
...,...
474,Asi-PAKI001.Esper.2007
475,Asi-PAKI014.Esper.2007
476,Ocn-TongueoftheOcean.Rosenheim.2005
477,Ocn-TongueoftheOcean.Rosenheim.2005


Find the missing records:

In [43]:
missing=[]

for index,row in df_pickle.iterrows():
    if (row['dataSetName']) in df['dsname'].values:
        pass
    else:
        missing.append(row['dataSetName'])

missing

['NAm-Bennington.Luckman.2001',
 'Asi-SouthChina.Wang.1998',
 'NAm-RedMountainPassSilverton.Graybill.1994',
 'NAm-BlanchardRiver.Luckman.2013',
 "NAm-WrangellsComposite.D'Arrigo.2006",
 'NAm-Pintlers.Littell.2011',
 'NAm-Sugarloaf.Kenigsberg.2013',
 'NAm-ShermanCreekPass.Briffa.1996',
 'NAm-ShermanCreekPass.Briffa.1996',
 'NAm-CanyonCreek.Lloyd.2002',
 'NAm-VicaryMine.Sauchyn.2016',
 "NAm-SouthernAlaskacomposite.D'Arrigo.2006",
 'Eur-LakeSilvaplana.Trachsel.2010',
 'Asi-Hunan-Jiangsu.Zhang.1980',
 'NAm-GeraldineLakes.Luckman.2001',
 'NAm-HellsKitchenLake.Gajewski.1988',
 'NAm-BowSummit-PeytoLake.Luckman.2005',
 'Ocn-XishaIslandSouthChinaSea.Sun.2004',
 'NAm-EurekaSummit.Schweingruber.1996',
 'NAm-EurekaSummit.Schweingruber.1996',
 'Ocn-RAPiD-12-1K.Thornalley.2009',
 'NAm-McGinnisTrail.Wiles.2013',
 'NAm-ConroyLake.Gajewski.1988',
 'Ocn-BunakenIsland.Charles.2003',
 'NAm-PeytoLake.Schweingruber.1996',
 'NAm-TamarackBowl.Sauchyn.2016',
 'NAm-SignalMountain.Luckman.2001',
 'NAm-CarltonRid